# Drake Scene with two UR5e robots

To run this notebook, you need access to the currently private `airo-planner` and install it.

In [ ]:
import os
from pathlib import Path
import numpy as np
from airo_planner.utils import files
from pydrake.geometry import Meshcat
from pydrake.geometry import Meshcat
from pydrake.math import RigidTransform

# from pydrake.visualization import ApplyVisualizationConfig, VisualizationConfig
from pydrake.geometry import MeshcatVisualizer
from pydrake.planning import RobotDiagramBuilder, SceneGraphCollisionChecker

In [ ]:
robot_diagram_builder = RobotDiagramBuilder()
scene_graph = robot_diagram_builder.scene_graph()
plant = robot_diagram_builder.plant()
builder = robot_diagram_builder.builder()
parser = robot_diagram_builder.parser()

# Add visualizer
meshcat = Meshcat()
visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)
# config = VisualizationConfig(publish_contacts=True, enable_alpha_sliders=True)
# ApplyVisualizationConfig(config, builder=builder, plant=plant, meshcat=meshcat)


# Load URDF files
resources_root = str(files.get_resources_dir())
ur5e_urdf = Path(os.path.join(resources_root, "robots", "ur5e", "ur5e.urdf"))
cube_urdf = "cube_and_cylinder.urdf"

arm_left_index = parser.AddModelFromFile(str(ur5e_urdf), model_name="arm_left")
arm_right_index = parser.AddModelFromFile(str(ur5e_urdf), model_name="arm_right")
cube_index = parser.AddModelFromFile(str(cube_urdf))

# Weld some frames together
world_frame = plant.world_frame()
arm_left_frame = plant.GetFrameByName("base_link", arm_left_index)
arm_right_frame = plant.GetFrameByName("base_link", arm_right_index)
cube_frame = plant.GetFrameByName("base_link", cube_index)

plant.WeldFrames(world_frame, arm_left_frame)
plant.WeldFrames(world_frame, arm_right_frame, RigidTransform([0.9, 0, 0]))
plant.WeldFrames(world_frame, cube_frame, RigidTransform([0, 0.5, 0]))

# plant.set_discrete_contact_solver(DiscreteContactSolver.kSap)
# plant.Finalize()

# Set up collision checking
diagram = robot_diagram_builder.Build()


# Not sure if this is needed
def _configuration_distance(q1, q2):
    return np.linalg.norm(q1 - q2)


collision_checker = SceneGraphCollisionChecker(
    model=diagram,
    robot_model_instances=[arm_left_index, arm_right_index],
    configuration_distance_function=_configuration_distance,
    edge_step_size=0.125,
)

# Create default contexts ~= state
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)
diagram.ForcedPublish(context)

In [ ]:
q = plant.GetPositions(plant_context, arm_left_index).tolist()
q

In [ ]:
q_all = plant.GetPositions(plant_context)
collision_checker.CheckConfigCollisionFree(q_all)

In [ ]:
q_new = q.copy()
q_new[1] = -np.pi / 2
plant.SetPositions(plant_context, arm_left_index, q_new)
plant.SetPositions(plant_context, arm_right_index, q_new)

diagram.ForcedPublish(context)

q_all = plant.GetPositions(plant_context)
collision_checker.CheckConfigCollisionFree(q_all)

In [ ]:
collision_checker.CheckConfigCollisionFree(np.zeros(12))